In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sim.dy.model_intv import ModelIntv
import sim.dy.keys as I
import numpy.random as rd

In [2]:
seed = 11667
n_collect = 200
cvs = np.concatenate([np.linspace(0, 0.2, 11), np.linspace(0.2, 0.5, 13)[1:]])
coverage0 = 0.2

folders = [
#     'dy_hi', 
#     'dy_lo',
    'dy_ladd'
]

In [3]:
def collect_vul(pars_post, coverages=np.linspace(0, 0.8, 33), alg='VSC'):    
    m = ModelIntv()

    mss = list()
    mss_stats = list()

    for i, pars in enumerate(pars_post):
        y1, pars = m.find_baseline(pars, 2022)

        for k in coverages:    
            _, (ms, ms_stats), _ = m.simulate_onward(y1, pars, 
                                                     intv={'FullACF': {'Coverage': k, 'ScreenAlg': alg}}, polished=True)

            ms_stats.update({'Key': i, 'Coverage': k})            
            mss_stats.append(ms_stats)
            
            ms = ms.assign(Key=i, Coverage=k)
            mss.append(ms)
            
    mss = pd.concat(mss).assign(Alg=alg)
    mss_stats = pd.DataFrame(mss_stats).assign(Alg=alg)
    return mss, mss_stats


In [4]:
for folder in folders:
    rd.seed(seed)
    
    print(folder)
    out_path = f'out/{folder}'
    pars_post = json.load(open(f'{out_path}/Post.json', 'r'))
    
    sel = rd.choice(list(range(len(pars_post))), n_collect)
    sel.sort()
    
    pars_post = [pars_post[i] for i in sel]

    mss0, mss_stats0 = collect_vul(pars_post, alg='VSC', coverages=cvs)
    mss1, mss_stats1 = collect_vul(pars_post, alg='Sy', coverages=cvs)
    mss2, mss_stats2 = collect_vul(pars_post, alg='SyCx', coverages=cvs)

    mss = pd.concat([mss0, mss1, mss2])
    mss_stats = pd.concat([mss_stats0, mss_stats1, mss_stats2])
    mss.to_csv(f'{out_path}/Sim_VulACF_budget.csv')
    mss_stats.to_csv(f'{out_path}/Sim_VulACF_budget_stats.csv')

dy_ladd


In [5]:
def collect_fu(pars_post, coverage=coverage0):
    intvs = {
        'Baseline': {},
        'Vul_0_0': {'FullACF': {'Coverage': coverage, 'FollowUp': 0, 'Duration': 0}},
        'Vul_2_3': {'FullACF': {'Coverage': coverage, 'FollowUp': 4, 'Duration': 2}},
        'Vul_2_6': {'FullACF': {'Coverage': coverage, 'FollowUp': 2, 'Duration': 2}},
        'Vul_3_3': {'FullACF': {'Coverage': coverage, 'FollowUp': 4, 'Duration': 3}},
        'Vul_3_6': {'FullACF': {'Coverage': coverage, 'FollowUp': 2, 'Duration': 3}},
    }
    
    
    m = ModelIntv()

    mss = list()
    mss_stats = list()

    for i, pars in enumerate(pars_post):
        y1, pars = m.find_baseline(pars, 2022)
        
        for key, intv in intvs.items():
            _, (ms, ms_stats), _ = m.simulate_onward(y1, pars, intv=intv, polished=True)

            ms_stats.update({'Key': i, 'Scenario': key})            
            mss_stats.append(ms_stats)
            
            ms = ms.assign(Key=i, Scenario=key)
            mss.append(ms)
            
    mss = pd.concat(mss).assign(Coverage = coverage)
    mss_stats = pd.DataFrame(mss_stats).assign(Coverage = coverage)
    return mss, mss_stats

In [6]:
for folder in folders:
    print(folder)
    out_path = f'out/{folder}'
    pars_post = json.load(open(f'{out_path}/Post.json', 'r'))

    sel = rd.choice(list(range(len(pars_post))), n_collect)
    sel.sort()
    pars_post = [pars_post[i] for i in sel]
    
    mss, mss_stats = collect_fu(pars_post, coverage=0.2)
    mss_stats = mss_stats.fillna(0)
    mss.to_csv(f'{out_path}/Sim_VulACF_followup.csv')
    mss_stats.to_csv(f'{out_path}/Sim_VulACF_followup_stats.csv')

dy_ladd


In [7]:
mss_stats

,Inc,Mor,ACF_Footfall,ACF_Screened,ACF_Yield,ACF_TP,ACF_Fl,ACF_Sl,ACF_TPT,ACF_fu_Footfall,...,ACF_Alt_Footfall,ACF_Alt_Screened,ACF_Alt_Yield,ACF_Alt_TP,ACF_Alt_Fl,ACF_Alt_Sl,ACF_Alt_TPT,Key,Scenario,Coverage
0,320521.705500,24350.660381,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,Baseline,0.2
1,307115.978118,22652.579765,5.069395e+06,5.048961e+06,22165.522936,13367.853520,13367.853520,0.000000,8797.669416,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,Vul_0_0,0.2
2,295599.943621,21230.684427,5.069395e+06,5.049222e+06,20867.806129,12636.472997,12071.140187,565.332810,8231.333132,3.595479e+06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,Vul_2_3,0.2
3,295623.246410,21232.155671,5.069395e+06,5.049316e+06,20842.874732,12638.000924,12072.601933,565.398991,8204.873808,1.799980e+06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,Vul_2_6,0.2
4,295584.216574,21229.448376,5.069395e+06,5.049182e+06,20878.251793,12635.643460,12070.348420,565.295040,8242.608333,4.392873e+06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,Vul_3_3,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,366346.368591,18273.326124,5.069395e+06,5.043525e+06,14784.659790,10087.946045,10087.946045,0.000000,4696.713745,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14,Vul_0_0,0.2
86,357820.896721,17568.042240,5.069395e+06,5.043875e+06,14171.827443,9983.484575,9479.165576,504.318999,4188.342868,3.607672e+06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14,Vul_2_3,0.2
87,357837.132940,17568.762119,5.069395e+06,5.043922e+06,14158.743105,9984.185606,9479.832656,504.352950,4174.557499,1.804970e+06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14,Vul_2_6,0.2
88,357814.555396,17567.821278,5.069395e+06,5.043854e+06,14177.486979,9983.278961,9478.970090,504.308872,4194.208018,4.408556e+06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14,Vul_3_3,0.2


In [3]:
def collect_fux(pars_post, coverage=coverage0, alg="VSC"):    
    m = ModelIntv()
    
    mss = list()
    mss_stats = list()

    for i, pars in enumerate(pars_post):
        y1, pars = m.find_baseline(pars, 2022)

        _, (ms, ms_stats), _ = m.simulate_onward(y1, pars, intv={}, polished=True)

        ms = ms.reset_index().assign(Key=i, FollowUp=0, Duration=0)
        mss.append(ms)
        
        ms_stats.update({'Key': i, 'Scenario': 'Baseline', 'FollowUp': 0, 'Duration': 0})
        mss_stats.append(ms_stats)
    
        for n_fu in list(range(7)): # + [24, 30, 36]:
            for dur in np.linspace(0, 4, 9):
                intv = {'FullACF': {'Coverage': coverage, 'FollowUp': n_fu, 'Duration': dur, 'ScreenAlg': alg}}
                
                _, (ms, ms_stats), _ = m.simulate_onward(y1, pars, intv=intv, polished=True)
                
                ms = ms.reset_index().assign(Key=i, FollowUp=n_fu, Duration=dur)
                mss.append(ms)

                ms_stats.update({'Key': i, 'Scenario': 'Intervention', 'FollowUp': n_fu, 'Duration': dur})
                mss_stats.append(ms_stats)
            
    mss = pd.concat(mss).assign(Coverage = coverage)
    mss_stats = pd.DataFrame(mss_stats).assign(Coverage = coverage)
    return mss, mss_stats

In [4]:
for folder in folders:
    print(folder)
    out_path = f'out/{folder}'
    pars_post = json.load(open(f'{out_path}/Post.json', 'r'))

    sel = rd.choice(list(range(len(pars_post))), n_collect)
    sel.sort()
    pars_post = [pars_post[i] for i in sel]
    
    mss, mss_stats = collect_fux(pars_post, coverage=coverage0)
    mss_stats = mss_stats.fillna(0)
    mss.to_csv(f'{out_path}/Sim_VulACF_fudur.csv')
    mss_stats.to_csv(f'{out_path}/Sim_VulACF_fudur_stats.csv')

dy_ladd


In [10]:
mss_stats

,Inc,Mor,ACF_Footfall,ACF_Screened,ACF_Yield,ACF_TP,ACF_Fl,ACF_Sl,ACF_TPT,ACF_fu_Footfall,...,ACF_Alt_Yield,ACF_Alt_TP,ACF_Alt_Fl,ACF_Alt_Sl,ACF_Alt_TPT,Key,Scenario,FollowUp,Duration,Coverage
0,270088.825418,20751.059788,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0,Baseline,0,0.0,0.2
1,260813.656656,19556.936463,5.069395e+06,5.050549e+06,40610.282947,10800.912938,10800.912938,0.000000,29809.370009,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0,Intervention,0,0.0,0.2
2,252956.288354,18547.212688,5.069395e+06,5.050851e+06,39694.926104,10414.643501,9911.979708,502.663793,29280.282603,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0,Intervention,0,0.5,0.2
3,252963.053164,18548.131061,5.069395e+06,5.050850e+06,39695.297402,10415.074165,9912.387519,502.686646,29280.223237,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0,Intervention,0,1.0,0.2
4,252962.641365,18548.063504,5.069395e+06,5.050850e+06,39695.258454,10415.031248,9912.346724,502.684524,29280.227206,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0,Intervention,0,1.5,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1275,157870.028854,17138.374230,5.069395e+06,5.059422e+06,17202.455892,8692.799794,8296.614365,396.185429,8509.656098,5.412142e+06,...,0.0,0.0,0.0,0.0,0.0,19,Intervention,6,2.0,0.2
1276,157863.851707,17137.532075,5.069395e+06,5.059399e+06,17222.735811,8692.092502,8295.938595,396.153906,8530.643310,6.076006e+06,...,0.0,0.0,0.0,0.0,0.0,19,Intervention,6,2.5,0.2
1277,157861.257315,17137.329220,5.069395e+06,5.059380e+06,17239.236014,8691.845737,8295.702656,396.143081,8547.390277,6.609227e+06,...,0.0,0.0,0.0,0.0,0.0,19,Intervention,6,3.0,0.2
1278,157859.644588,17137.429796,5.069395e+06,5.059365e+06,17252.942019,8691.904300,8295.759091,396.145209,8561.037719,7.045870e+06,...,0.0,0.0,0.0,0.0,0.0,19,Intervention,6,3.5,0.2
